In [302]:
import pandas as pd
from PIL import Image,ImageDraw,ImageFont
import numpy as np
import math

In [303]:
df = pd.read_csv("lime-robustness-all-score-outcome.csv")

In [304]:
toppct = 0.05

In [305]:
test_matrix_and_labels = pd.read_csv("32092_2014-04-11.csv")

In [306]:
n = len(test_matrix_and_labels)

In [307]:
cutoff = int(math.ceil( len(test_matrix_and_labels) * toppct ))

In [308]:
visdf = test_matrix_and_labels.sort_values(by="unit_score",ascending=False)

In [309]:
visdf.set_index("officer_id",drop=True,inplace=True)

In [310]:
tmp = list(np.repeat(1,cutoff))
tmp.extend(np.repeat(0,n-cutoff))

visdf['pred'] = tmp

In [311]:
boxwidth = 32 # adjustable but not recommended
gridheight = int(math.ceil( np.sqrt( n / 2 ) )) # ensuring 2:1 w:h 
gridwidth = gridheight * 2
px_w,px_h = gridwidth * boxwidth, gridheight * boxwidth
canvas = Image.new('RGB',(px_w,px_h),"black")

In [312]:
visdf['x'] = np.repeat(range(gridwidth-1,-1,-1),gridheight)[:n] # reversed range
tmp = list(range(gridheight-1,-1,-1)) * gridwidth # reversed range
visdf['y'] = tmp[:n]

In [313]:
chosen_officers = df.officer_id.unique()

In [314]:
"""
axis_labels = []
for i in reversed(visdf.index): # reversed bc PIL causes slight overlap; advantageous to plot this order    
    if i in chosen_officers:
        if visdf.outcome.loc[i]==1:
            fillcolor = "hsl(354,63%,56%)" # red
            if visdf.pred.loc[i]==1:
                fillcolor = "hsl(354,63%,28%)" # darker red if model says '1'
        else:
            fillcolor = "hsl(211,52%,56%)" # blue
            if visdf.pred.loc[i]==1:
                fillcolor = "hsl(211,52%,28%)" # darker blue if model says '1'
    else:
        fillcolor = "rgb(100,100,100)"
        
    xcoord = visdf.x.loc[i] * boxwidth
    ycoord = visdf.y.loc[i] * boxwidth

    if visdf.y.loc[i] == max(visdf.y):
        axis_labels.append(str(visdf.unit_score.loc[i])[1:4])

    bbox = [
        xcoord,
        ycoord,
        xcoord + boxwidth,
        ycoord + boxwidth
    ]

    draw = ImageDraw.Draw(canvas)
    draw.rectangle(bbox,fill=fillcolor,outline=None)
    
    if i in chosen_officers:
        draw.text((xcoord - 1 + 4, ycoord - 1 + boxwidth/4),str(i),fill="white")"""

'\naxis_labels = []\nfor i in reversed(visdf.index): # reversed bc PIL causes slight overlap; advantageous to plot this order    \n    if i in chosen_officers:\n        if visdf.outcome.loc[i]==1:\n            fillcolor = "hsl(354,63%,56%)" # red\n            if visdf.pred.loc[i]==1:\n                fillcolor = "hsl(354,63%,28%)" # darker red if model says \'1\'\n        else:\n            fillcolor = "hsl(211,52%,56%)" # blue\n            if visdf.pred.loc[i]==1:\n                fillcolor = "hsl(211,52%,28%)" # darker blue if model says \'1\'\n    else:\n        fillcolor = "rgb(100,100,100)"\n        \n    xcoord = visdf.x.loc[i] * boxwidth\n    ycoord = visdf.y.loc[i] * boxwidth\n\n    if visdf.y.loc[i] == max(visdf.y):\n        axis_labels.append(str(visdf.unit_score.loc[i])[1:4])\n\n    bbox = [\n        xcoord,\n        ycoord,\n        xcoord + boxwidth,\n        ycoord + boxwidth\n    ]\n\n    draw = ImageDraw.Draw(canvas)\n    draw.rectangle(bbox,fill=fillcolor,outline=None)

In [315]:
DIR = "/Users/damoncrockett/Desktop/chop-new/"

In [316]:
axis_labels = []
for i in reversed(visdf.index): # reversed bc PIL causes slight overlap; advantageous to plot this order    
    if visdf.y.loc[i] == max(visdf.y):
        axis_labels.append(str(visdf.unit_score.loc[i])[1:4])
    
    xcoord = visdf.x.loc[i] * boxwidth
    ycoord = visdf.y.loc[i] * boxwidth
    
    if i not in chosen_officers:
        
        fillcolor = "black"
        bbox = [
            xcoord,
            ycoord,
            xcoord + boxwidth,
            ycoord + boxwidth
        ]

        draw = ImageDraw.Draw(canvas)
        draw.rectangle(bbox,fill=fillcolor,outline=None)
        
    else:
        im = Image.open(DIR+str(i)+".png")
        im.thumbnail((32,32),Image.ANTIALIAS)
        canvas.paste(im,(xcoord,ycoord))

In [317]:
"""Add ticks and labels"""  
axis_labels.reverse() # because we reversed the plotting order above
plot_canvas_height = px_h + boxwidth*2
plot_canvas = Image.new('RGB',(px_w,plot_canvas_height),"black")
plot_canvas.paste(canvas,(0,0)) #pasting old canvas onto larger

tick_locs_x = [int( item * boxwidth + boxwidth / 2 ) for item in range(gridwidth-1,-1,-1)]
tick_locs_x = tick_locs_x[:len(axis_labels)]

draw = ImageDraw.Draw(plot_canvas)
for i in range(len(tick_locs_x)):
    draw.line([
            (tick_locs_x[i], plot_canvas_height - boxwidth - boxwidth / 2),
            (tick_locs_x[i], plot_canvas_height - boxwidth * 2)
        ],
              fill="rgb(100,100,100)"
             )
    draw.text(
        (tick_locs_x[i] - boxwidth / 4 - 2, plot_canvas_height - boxwidth - 10),
        axis_labels[i],
        fill="rgb(100,100,100)"
    )

In [318]:
plot_canvas.save("32092_2014-04-11_chosen_miniplot.png")